![image](car.jpeg)




# 🚀 Chatbot Prototype for "Car-ing is Sharing" using Hugging Face Transformers

Welcome to the prototype implementation of a multi-functional chatbot built for **Car-ing is Sharing**, an innovative car dealership and rental company. This bot leverages **pre-trained Large Language Models (LLMs)** from Hugging Face to perform a variety of customer-facing NLP tasks, including:

- Sentiment classification of car reviews
- English-to-Spanish translation
- Question answering
- Text summarization

---

## 📦 1. Environment Setup

Import required libraries and suppress unnecessary warnings from Transformers.




In [18]:
# Import necessary packages
import pandas as pd
import torch

from transformers import logging
logging.set_verbosity(logging.WARNING)

# 📂 2. Load Dataset

Read in the car reviews dataset car_reviews.csv, which contains customer reviews and sentiment labels.




In [19]:
car_reviews = pd.read_csv('car_reviews.csv', sep=';')

print(car_reviews.shape)
car_reviews.head()

(5, 2)


,Review,Class
0,I am very satisfied with my 2014 Nissan NV SL....,POSITIVE
1,The car is fine. It's a bit loud and not very ...,NEGATIVE
2,"My first foreign car. Love it, I would buy ano...",POSITIVE
3,I've come across numerous reviews praising the...,NEGATIVE
4,I've been dreaming of owning an SUV for quite ...,POSITIVE


# 🔍 3. View Sample Review
Preview the first review in the dataset.




In [20]:
car_reviews.Review.iloc[0]

'I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use. Camping, road trips, etc. We dont have any children so I store most of the seats in my warehouse. I wanted the passenger van for the rear air conditioning. We drove our van from Florida to California for a Cross Country trip in 2014. We averaged about 18 mpg. We drove thru a lot of rain and It was a very comfortable and stable vehicle. The V8 Nissan Titan engine is a 500k mile engine. It has been tested many times by delivery and trucking companies. This is why Nissan gives you a 5 year or 100k mile bumper to bumper warranty. Many people are scared about driving this van because of its size. But with front and rear sonar sensors, large mirrors and the back up camera. It is easy to drive. The front and rear sensors also monitor the front and rear sides of the bumpers making it easier to park close to objects. Our Nissan NV is a Tow Monster. It pulls our 5000 pound travel trailer l

# 💬 4. Sentiment Classification
Use a lightweight sentiment classifier to analyze all reviews in the dataset.




In [21]:
from transformers import pipeline

# Load a lightweight sentiment analysis pipeline
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Analyze sentiment for all car reviews
results = sentiment_analyzer(list(car_reviews.Review))
print(results)

[{'label': 'POSITIVE', 'score': 0.929397702217102}, {'label': 'POSITIVE', 'score': 0.8654271364212036}, {'label': 'POSITIVE', 'score': 0.9994640946388245}, {'label': 'NEGATIVE', 'score': 0.9935314059257507}, {'label': 'POSITIVE', 'score': 0.9986565113067627}]


# 📊 5. Evaluation Metrics
Calculate Accuracy and F1 Score to evaluate model performance.



In [22]:
import evaluate

accuracy = evaluate.load('accuracy')
# Store the model outputs in predicted_labels
predicted_labels = [res['label'] for res in results]

# Map 'POSITIVE' to 1 and 'NEGATIVE' to 0
label_map = {'POSITIVE': 1, 'NEGATIVE': 0}
predictions = [label_map[label] for label in predicted_labels]

# Extract ground truth labels and map to binary
references = [label_map[label] for label in car_reviews['Class']]

# Evaluate accuracy and F1 score
accuracy_result = accuracy.compute(predictions=predictions, references=references)
f1 = evaluate.load('f1')
f1_result = f1.compute(predictions=predictions, references=references, average='binary')

print("Accuracy:", accuracy_result)
print("F1 Score:", f1_result)

Accuracy: {'accuracy': 0.8}
F1 Score: {'f1': 0.8571428571428571}


# 🌐 6. English-to-Spanish Translation
Install SentencePiece and load a translation model to support Spanish-speaking customers.




In [23]:
%pip install sentencepiece

from transformers import pipeline

# Load a lightweight English-to-Spanish translation pipeline
translator = pipeline("translation_en_to_es", model="Helsinki-NLP/opus-mt-en-es")


Note: you may need to restart the kernel to use updated packages.


c:\Users\El-Wattaneya\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## ✂️ Extract First Two Sentences for Translation


In [24]:
import re

text = car_reviews.Review.iloc[0]
sentences = re.split(r'(?<=[.!?])\s+', text)
first_two = ' '.join(sentences[:2])
print(first_two)

I am very satisfied with my 2014 Nissan NV SL. I use this van for my business deliveries and personal use.


## 🔁 Translate and Store Output

In [25]:
print(translator(first_two))
translated_review = translator(first_two)[0]['translation_text']
translated_review

[{'translation_text': 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'}]


'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta camioneta para mis entregas de negocios y uso personal.'

## 📏 Evaluate Translation with BLEU Score
Load reference translations and compute BLEU score

In [26]:
with open('reference_translations.txt', 'r', encoding='utf-8') as f:
	reference_translations = [line.strip() for line in f if line.strip()]
	
reference_translations[:]  

['Estoy muy satisfecho con mi Nissan NV SL 2014. Utilizo esta camioneta para mis entregas comerciales y uso personal.',
 'Estoy muy satisfecho con mi Nissan NV SL 2014. Uso esta furgoneta para mis entregas comerciales y uso personal.']

In [27]:
BLEU = evaluate.load('bleu')

BLEU.features

[{'predictions': Value(dtype='string', id='sequence'),
  'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id='references')},
 {'predictions': Value(dtype='string', id='sequence'),
  'references': Value(dtype='string', id='sequence')}]

In [28]:
bleu_score= BLEU.compute(predictions=[translated_review], references=[reference_translations])
bleu_score

{'bleu': 0.7794483794144498,
 'precisions': [0.9090909090909091,
  0.8571428571428571,
  0.75,
  0.631578947368421],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0476190476190477,
 'translation_length': 22,
 'reference_length': 21}

# ❓ 7. Extractive Question Answering (QA)
Use a lightweight QA model to answer questions based on the second review

In [30]:
car_reviews.Review.iloc[1]

"The car is fine. It's a bit loud and not very powerful. On one hand, compared to its peers, the interior is well-built. The transmission failed a few years ago, and the dealer replaced it under warranty with no issues. Now, about 60k miles later, the transmission is failing again. It sounds like a truck, and the issues are well-documented. The dealer tells me it is normal, refusing to do anything to resolve the issue. After owning the car for 4 years, there are many other vehicles I would purchase over this one. Initially, I really liked what the brand is about: ride quality, reliability, etc. But I will not purchase another one. Despite these concerns, I must say, the level of comfort in the car has always been satisfactory, but not worth the rest of issues found."

In [33]:
QA_LLM = pipeline('question-answering', model="deepset/minilm-uncased-squad2")

question = "What did he like about the brand?"
context = car_reviews.Review.iloc[1]

result = QA_LLM(question=question, context=context)
answer = result['answer']
answer

model.safetensors:   0%|          | 0.00/133M [00:00<?, ?B/s]

Some weights of the model checkpoint at deepset/minilm-uncased-squad2 were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/107 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

'ride quality, reliability'

## 🧠 QA Result Object


In [34]:
result

{'score': 0.4773615002632141,
 'start': 569,
 'end': 594,
 'answer': 'ride quality, reliability'}

## 🔁 (Optional) Extract Multiple Answers by Sentence
Split the review into sentences and run QA per sentence for richer answers.




In [ ]:
from nltk import sent_tokenize

sentences = sent_tokenize(context)
answers = []

for sent in sentences:
    result = QA_LLM(question=question, context=sent)
    if result['score'] > 0.3:  
        answers.append(result['answer'])

answers = list(set(answers))  
print(answers)

['The car is fine', 'normal', 'ride quality, reliability']


# 📝 8. Summarization
Summarize the last review into ~50-55 tokens using a lightweight summarization model.


In [41]:
summarizer = pipeline("summarization", model="Falconsai/text_summarization")

last_review = car_reviews.Review.iloc[-1]
summarized_text = summarizer(last_review, max_length=55, min_length=50, do_sample=False)[0]['summary_text']
print(summarized_text)

config.json: 0.00B [00:00, ?B/s]

c:\Users\El-Wattaneya\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\El-Wattaneya\.cache\huggingface\hub\models--Falconsai--text_summarization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

the Nissan Rogue provides me with the desired SUV experience without burdening me with an exorbitant payment . I have hauled 12 bags of mulch in the back with the seats down and could have held more . To address this concern, I am


# 🧾 Final Notes
This prototype demonstrates how LLMs can be adapted for:

Automated review analysis

Multilingual support

Knowledge extraction

Text summarization

Built with 🤗 Hugging Face by Yahia Zakaria.

[GitHub](https://github.com/Yahia20) · [LinkedIn](www.linkedin.com/in/yahia-zakaria-a27384213) 